This is my final preprocessing notebook before modeling. Here, I'll start with trying out models' performances given various variable encoding strategies. I need to explore the possibility of feature reduction; I created multiple features in the previous notebook but am not yet sure if they'll be valuable in making predictions. I can begin to explore various models in the process of all this. 

Notebook on which this one builds: https://github.com/fractaldatalearning/Capstone2/blob/main/notebooks/preprocessing2_feature_engineering.ipynb

In [ ]:
import pandas as pd
import numpy as np
import os
from library.sb_utils import save_file

import random
import collections

from IPython.display import Audio
sound_file = './alert.wav'

In [ ]:
df = pd.read_csv('../data/processed/features_engineered.csv')
df.info()